In [1]:
import sys, os
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../../../slayerPytorch/src")

In [2]:
import slayerSNN as snn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dtw import dtw, accelerated_dtw
from numpy.linalg import norm
from joblib import Parallel, delayed
import torch
import copy

In [3]:
class ProcessData:
    def __init__(self, path, obj_name, obj_number):
        self.path = path
        self.obj_name = obj_name
        self.obj_number = obj_number
        self.WINDOW_SIZE = 4
        
        # prepare file names
        fpath_bio  = self.path + self.obj_name + '/' + self.obj_name + '_' + str(self.obj_number) + '_bio.csv'
        # read files
        df_bio = pd.read_csv(fpath_bio, index_col=False)
        df_bio.timestamp = df_bio.timestamp - df_bio.timestamp[0]
        df_bio_temp = df_bio[ (df_bio.timestamp >= 1) & (df_bio.timestamp <= 1+self.WINDOW_SIZE) ]
        # impute if small data
        if df_bio_temp.shape[0] == 399:
            df_bio_temp = df_bio_temp.append(df_bio.iloc[df_bio_temp.index[-1] + 1])
        self.df_bio = df_bio_temp
    def get_electrode_data(self):
        elctrd_cols = []
        for i in range(1,20):
            elctrd_cols.append('ele' + str(i))
        return self.df_bio[elctrd_cols].values
    def get_pac_data(self):
        pac_vals = []
        for i in range(1,23):
            pac_vals.append('pac' + str(i))
            
        count = 0
        pac_values = np.zeros(self.WINDOW_SIZE*100*22)
        for ind in self.df_bio[pac_vals].index:
            pac_values[count:count+22] = self.df_bio.loc[ind, pac_vals].values
            count += 22
        return pac_values
    def get_timestamp(self):
        return self.df_bio.timestamp.values

In [4]:
# electrodes
fpath = '../material_data_Feb/'

In [18]:
%%time
big_pac_data = np.zeros([21*50, 19, 400])
labels = []
count = 0
current_label = -1
for i in range(1,22):
    current_label += 1
    mat_name='mat'+str(i)
    print(mat_name)
    for j in range(1,51):
        mat_obj = ProcessData(fpath ,mat_name, j)
        big_pac_data[count,:,:] = mat_obj.get_electrode_data().T
        labels.append(current_label)
        count += 1

mat1
mat2
mat3
mat4
mat5
mat6
mat7
mat8
mat9
mat10
mat11
mat12
mat13
mat14
mat15
mat16
mat17
mat18
mat19
mat20
mat21
CPU times: user 11.3 s, sys: 323 ms, total: 11.7 s
Wall time: 12.8 s


In [20]:
big_pac_data.shape

(1050, 19, 400)

In [23]:
np.save('auxiliaries/electrodes.npy',big_pac_data)

In [5]:
# compute PAC

In [7]:
%%time
big_pac_data = np.zeros([21*50, 8800])
count = 0
current_label = -1
for i in range(1,22):
    mat_name='mat'+str(i)
    print(mat_name)
    for j in range(1,51):
        mat_obj = ProcessData(fpath ,mat_name, j)
        big_pac_data[count,:] = mat_obj.get_pac_data()
        count += 1

mat1
mat2
mat3
mat4
mat5
mat6
mat7
mat8
mat9
mat10
mat11
mat12
mat13
mat14
mat15
mat16
mat17
mat18
mat19
mat20
mat21
CPU times: user 4min 3s, sys: 1.62 s, total: 4min 5s
Wall time: 4min 5s


In [8]:
np.save('auxiliaries/pac.npy',big_pac_data)